# 1. Import Dependencies

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, multilabel_confusion_matrix
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# 2. Check the coordinates

In [ ]:
df_bench = pd.read_csv('./labeling/benchpress_coords.csv')
df_dead = pd.read_csv('./labeling/deadlift_coords.csv')
df_squat = pd.read_csv('./labeling/squat_coords.csv')

In [ ]:
df_bench.head()

In [ ]:
df_dead.head()

In [ ]:
df_squat.head()

# 3. Data Preprocessing & Data Normalization

In [ ]:
# Load data
bench_data = np.loadtxt("./labeling/benchpress_coords.csv", delimiter=",", dtype=str, skiprows=1)
labels = bench_data[:, 0]
bench_data = bench_data[:, 1:]  # Select all columns except the first column

# One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
labels = encoder.fit_transform(labels.reshape(-1, 1))

# Handle missing values
bench_data = np.nan_to_num(bench_data)

bench_data = bench_data.astype(float)

# Split data
bench_X_train, bench_X_test, bench_y_train, bench_y_test = train_test_split(bench_data, labels, test_size=0.25)

# Convert to tensor
bench_X_train = torch.tensor(bench_X_train)
bench_X_test = torch.tensor(bench_X_test)
bench_y_train = torch.tensor(bench_y_train)
bench_y_test = torch.tensor(bench_y_test)

# Convert to float
bench_X_train = bench_X_train.float()
bench_X_test = bench_X_test.float()
bench_y_train = bench_y_train.float()
bench_y_test = bench_y_test.float()

In [ ]:
# Load data
squat_data = np.loadtxt("./labeling/squat_coords.csv", delimiter=",", dtype=str, skiprows=1)
labels_squat = squat_data[:, 0]
squat_data = squat_data[:, 1:]  # Select all columns except the first column

# One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
labels_squat = encoder.fit_transform(labels_squat.reshape(-1, 1))

# Handle missing values
squat_data = np.nan_to_num(squat_data)

squat_data = squat_data.astype(float)

# Split data
squat_X_train, squat_X_test, squat_y_train, squat_y_test = train_test_split(squat_data, labels_squat, test_size=0.25)

# Convert to tensor
squat_X_train = torch.tensor(squat_X_train)
squat_X_test = torch.tensor(squat_X_test)
squat_y_train = torch.tensor(squat_y_train)
squat_y_test = torch.tensor(squat_y_test)

# Convert to float
squat_X_train = squat_X_train.float()
squat_X_test = squat_X_test.float()
squat_y_train = squat_y_train.float()
squat_y_test = squat_y_test.float()

In [ ]:
# Load data
deadlift_data = np.loadtxt("./labeling/deadlift_coords.csv", delimiter=",", dtype=str, skiprows=1)
labels_deadlift = deadlift_data[:, 0]
deadlift_data = deadlift_data[:, 1:]  # Select all columns except the first column

# One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
labels_deadlift = encoder.fit_transform(labels_deadlift.reshape(-1, 1))

# Handle missing values
deadlift_data = np.nan_to_num(deadlift_data)

deadlift_data = deadlift_data.astype(float)

# Split data
deadlift_X_train, deadlift_X_test, deadlift_y_train, deadlift_y_test = train_test_split(deadlift_data, labels_deadlift, test_size=0.25)

# Convert to tensor
deadlift_X_train = torch.tensor(deadlift_X_train)
deadlift_X_test = torch.tensor(deadlift_X_test)
deadlift_y_train = torch.tensor(deadlift_y_train)
deadlift_y_test = torch.tensor(deadlift_y_test)

# Convert to float
deadlift_X_train = deadlift_X_train.float()
deadlift_X_test = deadlift_X_test.float()
deadlift_y_train = deadlift_y_train.float()
deadlift_y_test = deadlift_y_test.float()

# 4. Construct Deep Learning Model

In [ ]:
bench_y_test.shape

In [ ]:
bench_y_train.shape

### Bench Press

In [ ]:
class DeepClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(DeepClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, output_size)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.dropout(self.relu(self.fc3(x)))
        x = self.fc4(x)
        x = self.softmax(x)
        return x

In [ ]:
# Build model
input_size = 132
output_size = 6
bench_model = DeepClassifier(input_size, output_size)

In [ ]:
bench_criterion = nn.BCELoss()
bench_optimizer = optim.Adam(bench_model.parameters(), lr=0.001)

In [ ]:
bench_train_losses = []
bench_test_losses = []

epochs = 1000

for epoch in range(epochs):
    bench_model.train()
    bench_optimizer.zero_grad()
    bench_outputs = bench_model(bench_X_train)
    bench_loss = bench_criterion(bench_outputs, bench_y_train)
    bench_loss.backward()
    bench_optimizer.step()
    bench_train_losses.append(bench_loss.item())
    
    bench_model.eval()
    with torch.no_grad():
        bench_test_outputs = bench_model(bench_X_test)
        bench_test_loss = bench_criterion(bench_test_outputs, bench_y_test)
        bench_test_losses.append(bench_test_loss.item())

plt.plot(range(1, epochs+1), bench_train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), bench_test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.show()

In [ ]:
bench_model.eval()
with torch.no_grad():
    test_outputs = bench_model(bench_X_test)
    test_loss = bench_criterion(test_outputs, bench_y_test)
    print(f'Test Loss: {test_loss.item()}')

In [ ]:
# Select samples from test dataset
num_samples_to_test = 5  # Number of samples to check
for i in range(num_samples_to_test):
    sample_index = i
    input_sample = bench_X_test[sample_index]  # Input data
    target_sample = bench_y_test[sample_index]  # Actual target (ground truth) data

    # Convert input data to tensor
    input_tensor = torch.tensor(input_sample, dtype=torch.float32)

    # Input to model to generate prediction
    bench_model.eval()
    with torch.no_grad():
        prediction = bench_model(input_tensor.unsqueeze(0))  # Add batch dimension using unsqueeze

    # Print prediction results
    print(f"Sample {sample_index + 1}:")
    print("Input data:", input_sample)
    print("Actual target data:", target_sample)
    print("Predicted data:", torch.argmax(prediction))  # Output predicted class (not probability)
    probabilities = prediction.squeeze().tolist()  # Convert tensor to list
    probabilities_str = [f"{prob:.4f}" for prob in probabilities]
    print("Prediction probabilities:", probabilities_str)
    print()

In [ ]:
bench_model.eval()
with torch.no_grad():
    predicted_probs = bench_model(bench_X_test)

threshold = 0.8  # Set threshold
predicted_labels = (predicted_probs > threshold).numpy().astype(int)

# Calculate accuracy
accuracy = accuracy_score(bench_y_test, predicted_labels)
print(f"Accuracy: {accuracy:.3f}")

# Calculate precision
precision = precision_score(bench_y_test, predicted_labels, average='micro')
print(f"Precision: {precision:.3f}")

# Calculate recall
recall = recall_score(bench_y_test, predicted_labels, average='micro')
print(f"Recall: {recall:.3f}")

# Calculate F1 score
f1 = f1_score(bench_y_test, predicted_labels, average='micro')
print(f"F1 score: {f1:.3f}")

In [ ]:
confusion_matrix = multilabel_confusion_matrix(bench_y_test, predicted_labels)

def plot_multilabel_confusion_matrix(confusion_matrix):
    num_classes = len(confusion_matrix)
    fig, axes = plt.subplots(nrows=num_classes, ncols=1, figsize=(8, 6 * num_classes))

    for i in range(num_classes):
        sns.heatmap(confusion_matrix[i], annot=True, cmap="Blues", fmt="d", ax=axes[i])
        axes[i].set_title(f"Class {i+1} Confusion Matrix")
        axes[i].set_xlabel("Predicted label")
        axes[i].set_ylabel("True label")
        axes[i].set_xticklabels(['True', 'False'])
        axes[i].set_yticklabels(['True', 'False'])

    plt.tight_layout()
    plt.show()

# Visualize multilabel confusion matrix
plot_multilabel_confusion_matrix(confusion_matrix)

### Squat

In [ ]:
# Build model
input_size = 132
output_size = 8
squat_model = DeepClassifier(input_size, output_size)
squat_criterion = nn.BCELoss()
squat_optimizer = optim.Adam(squat_model.parameters(), lr=0.001)

In [ ]:
squat_train_losses = []
squat_test_losses = []

epochs = 1000

for epoch in range(epochs):
    squat_model.train()
    squat_optimizer.zero_grad()
    squat_outputs = squat_model(squat_X_train)
    squat_loss = squat_criterion(squat_outputs, squat_y_train)
    squat_loss.backward()
    squat_optimizer.step()
    squat_train_losses.append(squat_loss.item())
    
    squat_model.eval()
    with torch.no_grad():
        squat_test_outputs = squat_model(squat_X_test)
        squat_test_loss = squat_criterion(squat_test_outputs, squat_y_test)
        squat_test_losses.append(squat_test_loss.item())

plt.plot(range(1, epochs+1), squat_train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), squat_test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.show()

In [ ]:
squat_model.eval()
with torch.no_grad():
    squat_test_outputs = squat_model(squat_X_test)
    squat_test_loss = squat_criterion(squat_test_outputs, squat_y_test)
    print(f'Test Loss: {squat_test_loss.item()}')

In [ ]:
# Select samples from test dataset
num_samples_to_test = 5  # Number of samples to check
for i in range(num_samples_to_test):
    sample_index = i
    input_sample = squat_X_test[sample_index]  # Input data
    target_sample = squat_y_test[sample_index]  # Actual target (ground truth) data

    # Convert input data to tensor
    input_tensor = torch.tensor(input_sample, dtype=torch.float32)

    # Input to model to generate prediction
    squat_model.eval()
    with torch.no_grad():
        prediction = squat_model(input_tensor.unsqueeze(0))  # Add batch dimension using unsqueeze

    # Print prediction results
    print(f"Sample {sample_index + 1}:")
    print("Input data:", input_sample)
    print("Actual target data:", target_sample)
    print("Predicted data:", torch.argmax(prediction))  # Output predicted class (not probability)
    probabilities = prediction.squeeze().tolist()  # Convert tensor to list
    probabilities_str = [f"{prob:.4f}" for prob in probabilities]
    print("Prediction probabilities:", probabilities_str)
    print()

In [ ]:
squat_model.eval()
with torch.no_grad():
    predicted_probs = squat_model(squat_X_test)

threshold = 0.8  # Set threshold
predicted_labels = (predicted_probs > threshold).numpy().astype(int)

# Calculate accuracy
accuracy = accuracy_score(squat_y_test, predicted_labels)
print(f"Accuracy: {accuracy:.3f}")

# Calculate precision
precision = precision_score(squat_y_test, predicted_labels, average='micro')
print(f"Precision: {precision:.3f}")

# Calculate recall
recall = recall_score(squat_y_test, predicted_labels, average='micro')
print(f"Recall: {recall:.3f}")

# Calculate F1 score
f1 = f1_score(squat_y_test, predicted_labels, average='micro')
print(f"F1 score: {f1:.3f}")

In [ ]:
confusion_matrix = multilabel_confusion_matrix(squat_y_test, predicted_labels)

def plot_multilabel_confusion_matrix(confusion_matrix):
    num_classes = len(confusion_matrix)
    fig, axes = plt.subplots(nrows=num_classes, ncols=1, figsize=(8, 6 * num_classes))

    for i in range(num_classes):
        sns.heatmap(confusion_matrix[i], annot=True, cmap="Blues", fmt="d", ax=axes[i])
        axes[i].set_title(f"Class {i+1} Confusion Matrix")
        axes[i].set_xlabel("Predicted label")
        axes[i].set_ylabel("True label")
        axes[i].set_xticklabels(['True', 'False'])
        axes[i].set_yticklabels(['True', 'False'])

    plt.tight_layout()
    plt.show()

# Visualize multilabel confusion matrix
plot_multilabel_confusion_matrix(confusion_matrix)

### Deadlift

In [ ]:
# Build model
input_size = 132
output_size = 8
deadlift_model = DeepClassifier(input_size, output_size)
deadlift_criterion = nn.BCELoss()
deadlift_optimizer = optim.Adam(deadlift_model.parameters(), lr=0.001)

In [ ]:
deadlift_train_losses = []
deadlift_test_losses = []

epochs = 1000

for epoch in range(epochs):
    deadlift_model.train()
    deadlift_optimizer.zero_grad()
    deadlift_outputs = deadlift_model(deadlift_X_train)
    deadlift_loss = deadlift_criterion(deadlift_outputs, deadlift_y_train)
    deadlift_loss.backward()
    deadlift_optimizer.step()
    deadlift_train_losses.append(deadlift_loss.item())
    
    deadlift_model.eval()
    with torch.no_grad():
        deadlift_test_outputs = deadlift_model(deadlift_X_test)
        deadlift_test_loss = deadlift_criterion(deadlift_test_outputs, deadlift_y_test)
        deadlift_test_losses.append(deadlift_test_loss.item())

plt.plot(range(1, epochs+1), deadlift_train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), deadlift_test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.show()

In [ ]:
deadlift_model.eval()
with torch.no_grad():
    deadlift_test_outputs = deadlift_model(deadlift_X_test)
    deadlift_test_loss = deadlift_criterion(deadlift_test_outputs, deadlift_y_test)
    print(f'Test Loss: {deadlift_test_loss.item()}')

In [ ]:
# Select samples from test dataset
num_samples_to_test = 5  # Number of samples to check
for i in range(num_samples_to_test):
    sample_index = i
    input_sample = deadlift_X_test[sample_index]  # Input data
    target_sample = deadlift_y_test[sample_index]  # Actual target (ground truth) data

    # Convert input data to tensor
    input_tensor = torch.tensor(input_sample, dtype=torch.float32)

    # Input to model to generate prediction
    deadlift_model.eval()
    with torch.no_grad():
        prediction = deadlift_model(input_tensor.unsqueeze(0))  # Add batch dimension using unsqueeze

    # Print prediction results
    print(f"Sample {sample_index + 1}:")
    print("Input data:", input_sample)
    print("Actual target data:", target_sample)
    print("Predicted data:", torch.argmax(prediction))  # Output predicted class (not probability)
    probabilities = prediction.squeeze().tolist()  # Convert tensor to list
    probabilities_str = [f"{prob:.4f}" for prob in probabilities]
    print("Prediction probabilities:", probabilities_str)
    print()

In [ ]:
deadlift_model.eval()
with torch.no_grad():
    predicted_probs = deadlift_model(deadlift_X_test)

threshold = 0.8  # Set threshold
predicted_labels = (predicted_probs > threshold).numpy().astype(int)

# Calculate accuracy
accuracy = accuracy_score(deadlift_y_test, predicted_labels)
print(f"Accuracy: {accuracy:.3f}")

# Calculate precision
precision = precision_score(deadlift_y_test, predicted_labels, average='micro')
print(f"Precision: {precision:.3f}")

# Calculate recall
recall = recall_score(deadlift_y_test, predicted_labels, average='micro')
print(f"Recall: {recall:.3f}")

# Calculate F1 score
f1 = f1_score(deadlift_y_test, predicted_labels, average='micro')
print(f"F1 score: {f1:.3f}")

In [ ]:
confusion_matrix = multilabel_confusion_matrix(deadlift_y_test, predicted_labels)

def plot_multilabel_confusion_matrix(confusion_matrix):
    num_classes = len(confusion_matrix)
    fig, axes = plt.subplots(nrows=num_classes, ncols=1, figsize=(8, 6 * num_classes))

    for i in range(num_classes):
        sns.heatmap(confusion_matrix[i], annot=True, cmap="Blues", fmt="d", ax=axes[i])
        axes[i].set_title(f"Class {i+1} Confusion Matrix")
        axes[i].set_xlabel("Predicted label")
        axes[i].set_ylabel("True label")
        axes[i].set_xticklabels(['True', 'False'])
        axes[i].set_yticklabels(['True', 'False'])

    plt.tight_layout()
    plt.show()

# Visualize multilabel confusion matrix
plot_multilabel_confusion_matrix(confusion_matrix)

# Save models

In [ ]:
import pickle

# Bench Press
with open('benchpress_model_deep.pkl', 'wb') as f:
    pickle.dump(bench_model, f)

# Squat
with open('squat_model_deep.pkl', 'wb') as f:
    pickle.dump(squat_model, f)

# Deadlift
with open('deadlift_model_deep.pkl', 'wb') as f:
    pickle.dump(deadlift_model, f)

In [ ]:
bench_X_test.shape

In [ ]:
squat_X_test.shape

In [ ]:
deadlift_X_test.shape